In [1]:
from tqdm import tqdm
from matplotlib import pyplot as plt
import time
from abc import abstractmethod
import numpy as np
from sklearn.linear_model import Lasso
from evaluator.evaluator import evaluator
from methods.our_method import Our_method

In [2]:
n_features=16
n_samples=100
n_packs=21
s=16
L=6
h=4
eval=evaluator(n_features, n_packs, n_samples, s, L)
model=Our_method(n_features, s, L)
_,model_sse,model_time=eval.t11_eval(model,h)

u:\文档\code\讨论班\11-14\simulation\methods\our_method.py:78: RuntimeWarning: overflow encountered in multiply
  grad=2*np.dot(X.T,np.dot(X,delta)-y)+lamb*delta/np.linalg.norm(delta)
u:\文档\code\讨论班\11-14\simulation\methods\our_method.py:78: RuntimeWarning: invalid value encountered in divide
  grad=2*np.dot(X.T,np.dot(X,delta)-y)+lamb*delta/np.linalg.norm(delta)


In [3]:
model_sse

nan